## Getting Reddit Comment Data

In [1]:
import praw
from praw.models import MoreComments
import tqdm
import pandas as pd
import statistics
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
reddit = praw.Reddit(
    client_id="p1dG7hgoowK4BSlUdar1WQ",
    client_secret="pEePtSnw7KMDZi6fCzkKaOth_pgKpQ",
    password="outdoortuesday",
    user_agent="Big Data by u/DISWillJayminMaya ",
    username="DISWillJayminMaya ",
)

In [3]:
analyzer = SentimentIntensityAnalyzer()

def returnListofScores(listofcomments):
    templist=[]
    for com in listofcomments:
        temp = analyzer.polarity_scores(com)
        compScore = temp['compound']
        templist.append(compScore)
    return templist


In [4]:
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']
subreddit = reddit.subreddit('wallstreetbets')

scoreDict={key:list() for key in tickerlist}

hot = subreddit.hot(limit=25) # getting first 15 posts in the 'hot' section of the subreddit
sum = [0] * len(tickerlist) # our output array
counttotal = 0 # total number of comment read
submissions_counter = 0

rel_comments = [] # List of comments that are relevant to the ticker list items

In [5]:
# Loop for fetching the comments and the amount of time each ticker is mentioned
for submissions in hot:
    if not submissions.stickied:
        submissions_counter+=1
        if submissions_counter > 5:
            comments = submissions.comments
            for comment in comments:
                if isinstance(comment, MoreComments):
                    continue
                counttotal+=1
                for i, ticker in enumerate(tickerlist):
                    if ticker in comment.body:
                        rel_comments.append(comment.body)
                        sum[i]=sum[i]+1
                        temp = returnCompScore(comment.body)
                        scoreDict[ticker].append(temp)

NameError: name 'returnCompScore' is not defined

In [6]:
meanDict={key:list() for key in tickerlist}
spreadDict={key:list() for key in tickerlist}

for tick in tickerlist:
    if len(scoreDict[tick])>0:
        meanDict[tick]=statistics.mean(scoreDict[tick])
    if len (scoreDict[tick])>1:
        spreadDict[tick]=statistics.stdev(scoreDict[tick])

In [7]:
print(meanDict)
print(spreadDict)

{'GME': [], 'Gamestop': [], 'SPY': [], 'TWTTR': [], 'Twitter': [], 'TSLA': [], 'Tesla': [], 'AMD': []}
{'GME': [], 'Gamestop': [], 'SPY': [], 'TWTTR': [], 'Twitter': [], 'TSLA': [], 'Tesla': [], 'AMD': []}


In [8]:
output=pd.DataFrame(data={'Tick': tickerlist, 'Counts': sum})
print('Total comments read: ',counttotal)
print(output[output['Counts']>0])

Total comments read:  249
  Tick  Counts
0  GME       1


## Getting Stock Information

In [3]:
import requests, os, json, csv

size = 'compact' # 'full' for complete historical data, 'compact' for most recent 100
ticker = ['GME', 'SPY', 'TWTTR', 'TSLA', 'AMD'] # stock tickers to search for
datatype = 'csv' # 'json' for JSON output, 'csv' for CSV output

for stock in ticker:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock}&outputsize={size}&datatype={datatype}&apikey=QC1C7LRPUTLC597Q'
    response = requests.get(url)
    #Save CSV to file
    with open(f'{stock}.csv', 'wb') as file:
        file.write(response.content)

## Charting Data

In [4]:
amd_data = pd.read_csv("AMD.csv")
amd_data

,timestamp,open,high,low,close,volume
0,2022-04-29,88.05,91.790,85.3800,85.52,82647701
1,2022-04-28,86.67,90.580,84.7800,89.64,91495449
2,2022-04-27,84.25,87.900,84.0200,84.91,83125054
3,2022-04-26,89.74,90.120,85.0800,85.16,87805574
4,2022-04-25,89.86,91.370,88.6100,90.69,93481042
...,...,...,...,...,...,...
95,2021-12-13,138.25,139.400,133.4150,133.80,42173963
96,2021-12-10,141.29,141.365,135.8200,138.55,42224275
97,2021-12-09,145.16,146.690,137.8000,138.10,53019926
98,2021-12-08,144.96,147.040,142.7000,145.24,40977478


get avg, spread, median of sentiment of comments in any given period
    compare this with the performance of the stock

model:
    based on last x days of reddit comments, what is the price going to be?
    take data from x days, put it all into one vector, and predict from this
    
    

## Making a function to return all comments that mention a stock based on a given date range

In [ ]:
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

def make_request(uri, max_retries = 5):
    """
    Function taken from medium article:
    https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
    """
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)

def get_intervals(startPOSTIX, endPOSTIX, daysInInterval = 1):
    """
    get_intervals goes day by day through the start and end dates, returning that day's POSTIX
    """
    # 86,400 seconds in a day:
    period = (86400 * daysInInterval)
    end = startPOSTIX + period
    
    yield(int(startPOSTIX), int(end))
    
    padding = 1
    while end <= endPOSTIX:
        startPOSTIX = end + padding
        end = (startPOSTIX - padding) + period
        yield int(startPOSTIX), int(end)
    
    
def pull_posts_for(subreddit, start_at, end_at):
    """
    Function taken from medium article:
    https://medium.com/@pasdan/how-to-scrap-reddit-using-pushshift-io-via-python-a3ebcc9b83f4
    """
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_'
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts(
        make_request(URI_TEMPLATE.format
                     (subreddit, start_at, end_at, SIZE))['data'])
    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - (10)
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format( \
                    subreddit, new_start_at, end_at, bSIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)
    return post_collections

def get_comments_by_date (startDate, endDate, subreddit='wallstreetbets'):
    """
    Takes a given time interval and scrapes the given subreddit for all of the comments
    that relate to the given ticker name, returning them as an array. Basic structure taken 
    from medium article.
    
    THIS CURRENTLY WILL NOT WORK IF GIVEN TODAY'S DATE. IT WILL ATTEMPT TO FETCH TOMORROW'S POSTS FOREVER
    """
    # Converting start and end dates to POSTIX:
    startDate = math.floor(startDate.timestamp())
    endDate = math.floor(endDate.timestamp())

    posts = []
    # This loop gets all of the posts in the given timeframe
    for interval in get_intervals(startDate, endDate):
        print("-- Fetching Posts From: ", datetime.fromtimestamp(interval[0]), " to ", datetime.fromtimestamp(interval[1]))
        pulled_posts = pull_posts_for(subreddit, interval[0], interval[1])
        posts.extend(pulled_posts)
        time.sleep(.100) # So as not to over request reddit

    TIMEOUT_SECS = 1
    
    reddit_posts = []
    reddit_comments = {}
    reddit_comments[startDate] = []
    
    # Going through each unique post and comment and adding them to the relevant arrays
    #  WARNING: only looking at first 100 posts of each day
    for sub_id in np.unique([post['id'] for post in posts])[:100]:
        # Only looking at posts with more than 100 upvotes to speed the process up
        if reddit.submission(sub_id).ups > 100:
            sub = reddit.submission(id=sub_id)
            reddit_posts.append(sub)
            sub.comments.replace_more(limit=None)
            # Looping through each comment:
            temp_com_count = 0
            for comment in sub.comments.list()[:100]: 
                temp_com_count += 1
                reddit_comments[startDate].append(comment.body)
                
            print("---- Fethced {} comments from post {}".format(temp_com_count, sub_id))
#             time.sleep(TIMEOUT_SECS)

    return reddit_comments

In [95]:
# Jan 11 - Jan 27 2021
date_range = []

# Filling array with dates (should be 11, 28 for GME Boom)
for i in range(11, 28):
    date_range.append(datetime(2021, 1, i))

In [112]:
all_comments = {}
for day in date_range:
    end_day = day
    start_day = day - timedelta(days=1)
    temp_all_coms = get_comments_by_date(start_day, end_day)
    all_comments.update(temp_all_coms)
    time.sleep(5)

-- Fetching Posts From:  2021-01-10 00:00:00  to  2021-01-11 00:00:00
-- Fetching Posts From:  2021-01-11 00:00:01  to  2021-01-12 00:00:00
--- Current Post ID:  ku2fx9
---- Current Comment ID:  gipi7uy
---- Current Comment ID:  gipjccd
---- Current Comment ID:  giq0m76
---- Current Comment ID:  gipjog0
---- Current Comment ID:  gipvxwc
---- Current Comment ID:  gipxcmx
---- Current Comment ID:  giqp8hu
---- Current Comment ID:  giptay7
---- Current Comment ID:  giq9osz
---- Current Comment ID:  giqgosu
--- Current Post ID:  ku2hgj
---- Current Comment ID:  gipizyq
---- Current Comment ID:  gipjqn1
---- Current Comment ID:  gipj3xf
---- Current Comment ID:  gipo0h2
---- Current Comment ID:  gipikpc
---- Current Comment ID:  giplbac
---- Current Comment ID:  gippzm5
---- Current Comment ID:  giq55tw
---- Current Comment ID:  gipigw2
---- Current Comment ID:  gipjfpl
--- Current Post ID:  ku2kna
---- Current Comment ID:  gipz0dv
---- Current Comment ID:  gipj4d4
---- Current Comment ID: 

KeyboardInterrupt: 

In [113]:
all_comments = {}

In [131]:
# Change this to the next day:
day = date_range[6]
end_day = day
start_day = day - timedelta(days=1)
temp_all_coms = get_comments_by_date(start_day, end_day)
all_comments.update(temp_all_coms)

-- Fetching Posts From:  2021-01-16 00:00:00  to  2021-01-17 00:00:00
-- Fetching Posts From:  2021-01-17 00:00:01  to  2021-01-18 00:00:00
--- Current Post ID:  ky68ce
---- Fethced 48 comments from post
--- Current Post ID:  ky69b4
---- Fethced 97 comments from post
--- Current Post ID:  ky6eze
---- Fethced 100 comments from post
--- Current Post ID:  ky6gr4
---- Fethced 49 comments from post
--- Current Post ID:  ky6lx7
---- Fethced 76 comments from post
--- Current Post ID:  ky6sdk
---- Fethced 23 comments from post
--- Current Post ID:  ky6u4h
---- Fethced 37 comments from post
--- Current Post ID:  ky6u6s
---- Fethced 65 comments from post
--- Current Post ID:  ky6w9m
---- Fethced 58 comments from post
--- Current Post ID:  ky6ycl
---- Fethced 78 comments from post


In [132]:
all_comments.keys()

dict_keys([1610233200, 1610319600, 1610492400, 1610578800, 1610751600])

In [133]:
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']
ticker_dict = {}

# Filling ticker_dict with empty dictionaries
for tick in tickerlist:
    ticker_dict[tick] = {}

    # Filling the dictionaries in ticker_dict with empty lists
for tick in tickerlist:
    for key in all_comments.keys():
        ticker_dict[tick][key] = []
        
# Adding the comments to their ticker and date
for tick in tickerlist:
    for key in all_comments.keys():
        for com in all_comments[key]:
            if tick in com:
                ticker_dict[tick][key].append(com)

In [134]:
all_comments.keys()

dict_keys([1610233200, 1610319600, 1610492400, 1610578800, 1610751600])

In [135]:
ticker_dict

{'GME': {1610233200: ["Mr. Beast definitely has the funds to take a nice stake in GME. Wonder if he's a gambling man? Oh wait, he just bought a **million** dollars worth of scratch off tickets. Someone tweet the man the DD."],
  1610319600: ['I will become a millionaire by the end of 2021. I will escape the rat race. And I will lose it all on a YOLO GME put \n\n&#x200B;\n\np: $CMC 88 contracts 26.51 call'],
  1610492400: ['Inorganic brand engagement: An autistic way\n\nI see, people creating memes and post to encourage herd buying of GME shares. In fact, I joined in myself. \n\nI was thinking if people could boost brand engagements of a company in a similar way. The ticker is GOEV. What if 10-20% people on this sub decided to follow and engage with Canoo Inc’s social media accounts/posts e.g. following their page, liking and commenting their posts. This could provide a 5-10 fold increase in their engagement rate. Better brand engagement would mean better awareness of the company and th

In [139]:
comment_data = pd.DataFrame.from_dict(ticker_dict)
pd.DataFrame.to_csv('comment_data.csv', comment_data)

AttributeError: 'str' object has no attribute 'to_frame'

## Making the Comments Data Frame

In [ ]:
comment_df = pd.DataFrame()
tickerlist = ['GME', 'Gamestop', 'SPY', 'TWTTR', 'Twitter', 'TSLA', 'Tesla', 'AMD']
# all_comments = get_comments_by_date(start, end)
ticker_dates_dict = {}

for tick in tickerlist:
    ticker_dates_dict[tick] = pd.DataFrame()

# Dates begin yesterday through today:
end_day = datetime.today() # End day is today
start_day = end_day - timedelta(days=7) # Start day is one week ag

# Running for the past week:
for tick in tickerlist:
    for i in range(0, 7):
        end_day = datetime.today() - timedelta(days=i)
        start_day = datetime.today() - timedelta(days=i + 1)

In [ ]:
ticker_dates_dict['SPY']